# Y E A S T

### 'fine replicates'

In [ ]:
import pandas as pd
import numpy as np
import scipy.special as sc
from scipy.stats import entropy, wasserstein_distance
import matplotlib.pyplot as plt
import matplotlib as m
from sklearn.feature_selection import f_regression, mutual_info_regression

In [ ]:
# basic bayexpress functions

# assuming a flat prior (for now?)
u_1 = 1
u_2 = 1

# calculating Bayes factors
def get_BF(N_1, n_1, N_2, n_2):

    return (sc.betaln( u_1 + n_1, u_2 + N_1 - n_1) + sc.betaln( u_1 + n_2, u_2 + N_2 - n_2) - sc.betaln( u_1 + n_1 + n_2, u_2 + N_1 - n_1 + N_2 - n_2)) / np.log(10) 

# ratio of expression 
# probably best to make it log2 to fit with most other software

def get_FC(N_1, n_1, N_2, n_2):
    rate_1 = (u_1 + n_1) / (u_2 + N_1 - n_1)
    rate_2 = (u_1 + n_2) / (u_2 + N_2 - n_2)

    return np.log2(rate_2 / rate_1)


# assuming a flat prior (for now?)
u_1 = 1
u_2 = 1

# calculating Bayes factors for consistency checks

def get_BF_IC(data):
    # this range is irrelevant if we want to do all 
    k = len(data.columns)

    evidence2 = np.full(len(data), 0)

    # iterating over j until k
    for col in data.columns[1:k]: 
        n_j = data[col]
        # print(n_j, 'n_j')
        N_j = sum(data[col])
        # print(N_j, 'N_j')
        evidence2 = evidence2 + sc.betaln(u_1 + n_j, u_2 + N_j - n_j)

    N = sum(data.iloc[:,1:k].sum(axis=0, numeric_only=True))
    n_i = data.iloc[:,1:k].sum(axis=1, numeric_only=True)

    # print(n_i, 'n_i')
    # print(N, 'N')

    evidence1 = sc.betaln( u_1 + n_i, u_2 + N - n_i)

    return (evidence2 - evidence1) / np.log(10) 


In [ ]:
WT_yeast = pd.read_csv('WT_yeast.csv', index_col=0)
Snf2_yeast = pd.read_csv('Snf2_yeast.csv', index_col=0)

display(WT_yeast)
display(Snf2_yeast)

In [ ]:

# assuming a flat prior (for now?)
u_1 = 1
u_2 = 1

# calculating Bayes factors for k number of reps

def get_BF(data):
    for k in range(3,43):

        evidence2 = np.full(len(data), 0)

        # iterating over j until k
        for col in data.columns[1:k]: 
            n_j = data[col]
            # print(n_j, 'n_j')
            N_j = sum(data[col])
            # print(N_j, 'N_j')
            evidence2 = evidence2 + sc.betaln(u_1 + n_j, u_2 + N_j - n_j)

        N = sum(data.iloc[:,1:k].sum(axis=0, numeric_only=True))
        n_i = data.iloc[:,1:k].sum(axis=1, numeric_only=True)

        # print(n_i, 'n_i')
        # print(N, 'N')

        evidence1 = sc.betaln( u_1 + n_i, u_2 + N - n_i)
        
        data[f'BF_{k-1}'] = (evidence2 - evidence1) / np.log(10) 

    return data


# calculating average <q> for each additional replicate for the plots


def get_avq(data):
    output = pd.DataFrame({'genes': data.genes})
    for k in range(2,len(data.columns)):

        N = sum(data.iloc[:,1:k].sum(axis=0, numeric_only=True))

        n_i = data.iloc[:,1:k].sum(axis=1, numeric_only=True)
        
        output[f'{k-1}'] = (n_i + 1) / (N+2)

    return output


# WT_yeast = get_BF(WT_yeast)

# display(WT_yeast)

# Snf2_yeast = get_BF(Snf2_yeast)

# display(Snf2_yeast)

WT_yeast_avq = get_avq(WT_yeast)

display(WT_yeast_avq)


Snf2_yeast_avq = get_avq(Snf2_yeast)

display(Snf2_yeast_avq)


In [ ]:
WT_yeast.sum(axis=0, numeric_only=True).describe()

In [ ]:
WT_yeast.describe().iloc[:,:3]

In [ ]:
WT_yeast_q = pd.DataFrame({})

for col in WT_yeast.columns[1:43]:

    WT_yeast_q[col+'_q'] = (WT_yeast[col]+1) / (sum(WT_yeast[col])+2)

WT_yeast_q

In [ ]:
Snf2_yeast_q = pd.DataFrame({})

for col in Snf2_yeast.columns[1:43]:

    Snf2_yeast_q[col+'_q'] = (Snf2_yeast[col]+1) / (sum(Snf2_yeast[col])+2)

Snf2_yeast_q

In [ ]:
WT_yeast_q.describe()

In [ ]:
WT_yeast_q_stats = WT_yeast_q.transpose().describe()

display(WT_yeast_q_stats)


# ------------

fig, ax = plt.subplots(dpi=300)

ax.grid()
ax.scatter(WT_yeast_q_stats.loc['mean'], WT_yeast_q_stats.loc['std'],
            
c='#332288', s=30, 
alpha=0.3, edgecolors='none')


plt.show()

# ------------

fig, ax = plt.subplots(dpi=300)

ax.grid()

ax.set_xscale("log", base=10)
ax.set_yscale("log", base=10)

ax.set_xlabel('Mean q_i')
ax.set_ylabel('Standard deviation q_i')

ax.scatter(WT_yeast_q_stats.loc['mean'], WT_yeast_q_stats.loc['std'],
            
c='#332288', s=30,
alpha=0.3, edgecolors='none')


plt.show()

# middle one exported as variability_WT.png for manuscript

In [ ]:
Snf2_yeast_q_stats = Snf2_yeast_q.transpose().describe()

display(Snf2_yeast_q_stats)


# ------------

fig, ax = plt.subplots(dpi=300)

ax.grid()
ax.scatter(Snf2_yeast_q_stats.loc['mean'], Snf2_yeast_q_stats.loc['std'],
            
c='#117733', s=30, 
alpha=0.3, edgecolors='none')


plt.show()

# ------------

fig, ax = plt.subplots(dpi=300)

ax.grid()

ax.set_xscale("log", base=10)
ax.set_yscale("log", base=10)

ax.set_xlabel('Mean q_i')
ax.set_ylabel('Standard deviation q_i')

ax.scatter(Snf2_yeast_q_stats.loc['mean'], Snf2_yeast_q_stats.loc['std'],
            
c='#117733', s=30,
alpha=0.3, edgecolors='none')



# middle one exported as variability_Snf2.png for manuscript

In [ ]:
# importing results from differential gene expression anaylsis
RALL_bayexpress = pd.read_csv('RALL_bayexpress.csv', index_col=0)

display(RALL_bayexpress)

In [ ]:
# 'volcano plots' =  BF vs. iFC

def get_denstiy_volcano(x, y):
    
    fig, ax = plt.subplots(figsize=(7,5), dpi=300)

    plt.hist2d( x, y, bins=50, norm=m.colors.LogNorm(), cmap='plasma', 
                # range=[[1, 1100], [0.0, 0.0105]], 
                alpha=0.9)

    plt.grid(color='black', linestyle='-', linewidth=0.7, alpha=0.5)
    
    cbar = plt.colorbar()

    cbar.set_label('# of genes')

    plt.xlabel('log10 Bayes factors')
    plt.ylabel('inferred log2 fold change')

    plt.show()

# ------------

fig, ax = plt.subplots(dpi=300)

ax.grid()
ax.scatter(RALL_bayexpress['BF'], RALL_bayexpress['FC'],
            
c='blue', s=10, 
alpha=0.3, edgecolors='none')


plt.show()

# ------------


get_denstiy_volcano(RALL_bayexpress['BF'], RALL_bayexpress['FC'])
get_denstiy_volcano(RALL_bayexpress.sort_values(by='BF')[:-100]['BF'], RALL_bayexpress.sort_values(by='BF')[:-100]['FC'])
get_denstiy_volcano(RALL_bayexpress.sort_values(by='BF')[:-1000]['BF'], RALL_bayexpress.sort_values(by='BF')[:-1000]['FC'])

# ------------

fig, ax = plt.subplots(dpi=300)

ax.grid()
ax.scatter(RALL_bayexpress.sort_values(by='BF')[:-1000]['BF'], RALL_bayexpress.sort_values(by='BF')[:-1000]['FC'],
            
c='blue', s=10, 
alpha=0.3, edgecolors='none')


plt.show()

# ------------


In [ ]:
results_RALL = pd.read_csv('results_RALL.csv', index_col=0)

results_RALL

### Examples for the paper

In [ ]:
# lowest BF

display(RALL_bayexpress.sort_values(by='BF')[:5])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[:5].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[:5].index])

print(list(RALL_bayexpress.sort_values(by='BF')[:5].locus_name))


In [ ]:
# low, but not lowest BF

display(RALL_bayexpress.sort_values(by='BF')[1000:1005])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[1000:1005].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[1000:1005].index])

print(list(RALL_bayexpress.sort_values(by='BF')[1000:1005].locus_name))


In [ ]:
# highest BF

display(RALL_bayexpress.sort_values(by='BF')[-5:])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[-5:].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[-5:].index])

print(list(RALL_bayexpress.sort_values(by='BF')[-5:].locus_name))


In [ ]:
# high, but not highest BF

display(RALL_bayexpress.sort_values(by='BF')[5300:5305])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[5300:5305].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[5300:5305].index])

print(list(RALL_bayexpress.sort_values(by='BF')[5300:5305].locus_name))


In [ ]:
# high, but not highest BF 2

display(RALL_bayexpress.sort_values(by='BF')[4000:4005])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[4000:4005].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[4000:4005].index])

print(list(RALL_bayexpress.sort_values(by='BF')[4000:4005].locus_name))


In [ ]:
# examples to discuss ...

display(RALL_bayexpress.sort_values(by='BF')[2000:2005])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[2000:2005].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[2000:2005].index])

print(list(RALL_bayexpress.sort_values(by='BF')[2000:2005].locus_name))

In [ ]:
# around 0

display(RALL_bayexpress.sort_values(by='BF')[1600:1605])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[1600:1605].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[1600:1605].index])

print(list(RALL_bayexpress.sort_values(by='BF')[1600:1605].locus_name))


In [ ]:
# where do the divisions get nice?

display(RALL_bayexpress.sort_values(by='BF')[1900:1905])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[1900:1905].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[1900:1905].index])

print(list(RALL_bayexpress.sort_values(by='BF')[1900:1905].locus_name))


In [ ]:
# low, but not lowest BF

display(RALL_bayexpress.sort_values(by='BF')[1000:1005])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[1000:1005].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='BF')[1000:1005].index])

print(list(RALL_bayexpress.sort_values(by='BF')[1000:1005].locus_name))


In [ ]:
# highest FC

display(RALL_bayexpress.sort_values(by='FC')[-5:])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='FC')[-5:].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='FC')[-5:].index])

print(list(RALL_bayexpress.sort_values(by='FC')[-5:].locus_name))


In [ ]:
# fold change around 0

display(RALL_bayexpress.sort_values(by='FC')[2000:2005])

display(WT_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='FC')[2000:2005].index])
display(Snf2_yeast.set_index('genes').iloc[RALL_bayexpress.sort_values(by='FC')[2000:2005].index])

print(list(RALL_bayexpress.sort_values(by='FC')[2000:2005].locus_name))


In [ ]:
# taking all replicates into account, which genes are only identified by bayexpress and not the other 2 packages?

# Criteria (BF > 1) & (FC > 1), 138 genes

# in collaboration with package_comparison_RALL.ipynb

display(RALL_bayexpress.set_index('locus_name').loc[['YAL016C-B', 'YAL031W-A', 'YAR035W', 'YAR068W', 'YBR018C']])

display(WT_yeast.set_index('genes').loc[['YAL016C-B', 'YAL031W-A', 'YAR035W', 'YAR068W', 'YBR018C']])
display(Snf2_yeast.set_index('genes').loc[['YAL016C-B', 'YAL031W-A', 'YAR035W', 'YAR068W', 'YBR018C']])

display(RALL_bayexpress.set_index('locus_name').loc[['YAL016C-B', 'YAL031W-A', 'YAR035W', 'YAR068W', 'YBR018C']])

In [ ]:
# taking all replicates into account, which genes are only identified by bayexpress and not the other 2 packages?

# Criteria (BF > 1) & (FC > 2)

# in collaboration with package_comparison_RALL.ipynb

display(RALL_bayexpress.set_index('locus_name').loc[['YCL048W', 'YFL058W', 'YGL034C', 'YHR007C-A', 'YIR017C', 'YJL077C',
       'YJR078W', 'YJR095W', 'YKR039W', 'YLR307W', 'YMR323W', 'YOL166C',
       'YOR100C', 'YPR077C', 'tM(CAU)P']])

display(WT_yeast.set_index('genes').loc[['YCL048W', 'YFL058W', 'YGL034C', 'YHR007C-A', 'YIR017C', 'YJL077C',
       'YJR078W', 'YJR095W', 'YKR039W', 'YLR307W', 'YMR323W', 'YOL166C',
       'YOR100C', 'YPR077C', 'tM(CAU)P']])
display(Snf2_yeast.set_index('genes').loc[['YCL048W', 'YFL058W', 'YGL034C', 'YHR007C-A', 'YIR017C', 'YJL077C',
       'YJR078W', 'YJR095W', 'YKR039W', 'YLR307W', 'YMR323W', 'YOL166C',
       'YOR100C', 'YPR077C', 'tM(CAU)P']])

display(RALL_bayexpress.set_index('locus_name').loc[['YCL048W', 'YFL058W', 'YGL034C', 'YHR007C-A', 'YIR017C', 'YJL077C',
       'YJR078W', 'YJR095W', 'YKR039W', 'YLR307W', 'YMR323W', 'YOL166C',
       'YOR100C', 'YPR077C', 'tM(CAU)P']])

print(['YCL048W', 'YFL058W', 'YGL034C', 'YHR007C-A', 'YIR017C', 'YJL077C',
       'YJR078W', 'YJR095W', 'YKR039W', 'YLR307W', 'YMR323W', 'YOL166C',
       'YOR100C', 'YPR077C', 'tM(CAU)P'])

In [ ]:
# taking all replicates into account
# which ones are positive in DESeq2 and edgeR but not bayexpress?

# Criteria (BF > 1) & (FC > 2)

# in collaboration with package_comparison_RALL.ipynb

display(RALL_bayexpress.set_index('locus_name').loc[['YAL061W', 'YBL005W-B', 'YBR012W-B', 'YDR055W', 'YDR098C-B',
       'YDR210C-D', 'YDR406W', 'YER160C', 'YFR053C', 'YFR055W', 'YGR161C-D',
       'YHR214C-B', 'YIL119C', 'YKL062W', 'YKL109W', 'YLL053C', 'YLR035C-A',
       'YLR058C', 'YLR258W', 'YLR327C', 'YLR346C', 'YNL284C-B', 'YOL155C',
       'YOR142W-B', 'YOR273C', 'YPR149W']])

display(WT_yeast.set_index('genes').loc[['YAL061W', 'YBL005W-B', 'YBR012W-B', 'YDR055W', 'YDR098C-B',
       'YDR210C-D', 'YDR406W', 'YER160C', 'YFR053C', 'YFR055W', 'YGR161C-D',
       'YHR214C-B', 'YIL119C', 'YKL062W', 'YKL109W', 'YLL053C', 'YLR035C-A',
       'YLR058C', 'YLR258W', 'YLR327C', 'YLR346C', 'YNL284C-B', 'YOL155C',
       'YOR142W-B', 'YOR273C', 'YPR149W']])
display(Snf2_yeast.set_index('genes').loc[['YAL061W', 'YBL005W-B', 'YBR012W-B', 'YDR055W', 'YDR098C-B',
       'YDR210C-D', 'YDR406W', 'YER160C', 'YFR053C', 'YFR055W', 'YGR161C-D',
       'YHR214C-B', 'YIL119C', 'YKL062W', 'YKL109W', 'YLL053C', 'YLR035C-A',
       'YLR058C', 'YLR258W', 'YLR327C', 'YLR346C', 'YNL284C-B', 'YOL155C',
       'YOR142W-B', 'YOR273C', 'YPR149W']])

display(RALL_bayexpress.set_index('locus_name').loc[['YAL061W', 'YBL005W-B', 'YBR012W-B', 'YDR055W', 'YDR098C-B',
       'YDR210C-D', 'YDR406W', 'YER160C', 'YFR053C', 'YFR055W', 'YGR161C-D',
       'YHR214C-B', 'YIL119C', 'YKL062W', 'YKL109W', 'YLL053C', 'YLR035C-A',
       'YLR058C', 'YLR258W', 'YLR327C', 'YLR346C', 'YNL284C-B', 'YOL155C',
       'YOR142W-B', 'YOR273C', 'YPR149W']])

print(['YAL061W', 'YBL005W-B', 'YBR012W-B', 'YDR055W', 'YDR098C-B',
       'YDR210C-D', 'YDR406W', 'YER160C', 'YFR053C', 'YFR055W', 'YGR161C-D',
       'YHR214C-B', 'YIL119C', 'YKL062W', 'YKL109W', 'YLL053C', 'YLR035C-A',
       'YLR058C', 'YLR258W', 'YLR327C', 'YLR346C', 'YNL284C-B', 'YOL155C',
       'YOR142W-B', 'YOR273C', 'YPR149W'])

In [ ]:
WT_yeast

In [ ]:
sc.kl_div(list(WT_yeast.iloc[0,1:]), list(Snf2_yeast.iloc[0,3:]))

In [ ]:
wasserstein_distance(list(WT_yeast.iloc[0,1:]), list(Snf2_yeast.iloc[0,3:]))

In [ ]:
RALL_bayexpress

In [ ]:
RALL_bayexpress['W'] = [wasserstein_distance(list(WT_yeast.iloc[i,1:]), list(Snf2_yeast.iloc[i,3:])) for i in range(len(RALL_bayexpress))]

RALL_bayexpress

In [ ]:

fig, ax = plt.subplots(dpi=300)

ax.grid()
ax.scatter(RALL_bayexpress['W'], RALL_bayexpress['BF'],
            
c='blue', s=10, 
alpha=0.3, edgecolors='none')

ax.set_xscale("log", base=10)

plt.show()

In [ ]:

fig, ax = plt.subplots(dpi=300)

ax.grid()
ax.scatter(RALL_bayexpress.sort_values(by='BF')[:-500]['W'], RALL_bayexpress.sort_values(by='BF')[:-500]['BF'],
            
c='blue', s=10, 
alpha=0.3, edgecolors='none')

ax.set_xscale("log", base=10)


plt.show()

In [ ]:
RALL_bayexpress.sort_values(by='W')

In [ ]:
RALL_bayexpress.loc[RALL_bayexpress.BF > 1]